# **Personalized Product Recommendations with ML PySpark and OpenAI Embeddings**

> This Project is designed to:  
- **build scalable recommendation systems** to enhance customer experiences and drive sales.  
- Construct a **data processing pipeline** using **PySpark**.  
- Implement **K-means clustering** with **OpenAI text embeddings**.  
- Develop a **recommendation system** that suggests products based on user behavior.  



### Task 1 - Set up the project

Installing the needed modules.

In [1]:
!pip install openai==1.16.2 python-dotenv pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


Imporint the modules

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType

from pyspark.ml.feature import VectorAssembler, PCA
from pyspark.ml.clustering import KMeans
import plotly.express as px

Setup the OpenAI API

Create a Spark session

Loading the dataset

List of 8 products recently viewed by the user.

In [ ]:
recently_viewed_products = [
    'P316',
    'P333',
    'P1115',
    'P1691',
    'P1082',
    'P397',
    'P1441',
    'P1054',
]

### Task 2 - Prepare the dataset

Combine `title` and `description` Columns

get the combined_text column and convert it into a list

Use OpenAI text embedding model to create the vector embeddings.

Let't put the embedding vectors into our original dataframe

Convert embedding vectors list into a Pyspark DataFrame

In [ ]:
features_column_names = [f"embedding_{i}" for i in range(len(embedding_vectors[0]))]

Add unique `row_id` to each row in the pysaprk dataframe

Add unique `row_id` to each row in our main pyspark dataframe `df`

Let's join the two dataframes

### Task 3 - Cluster products using K-means

Assemble the 512 Embedding Columns into a Single 'features' Column

Apply K-Means Clustering with 5 Clusters on the `features` Column

### Task 4 - Visualize the clusters

Let's reduce the dimensionality of our features for visualization purpose

`512 dimensions => 2 dimensions`

Let's plot the Clusters

In [ ]:
def plot_clusters(pca_df, num_clusters=5):
    """
    Plots a 2D visualization of clusters using Plotly Express.

    Parameters:
    - pca_df (DataFrame): A Pandas DataFrame containing columns 'x', 'y', and 'cluster'.
      'x' and 'y' are the 2D PCA components, and 'cluster' indicates the cluster label.
    - num_clusters (int): The number of unique clusters to display.
    - recently_viewed_df (DataFrame, optional): DataFrame with 'x' and 'y' coordinates for recently viewed products.

    This function creates an interactive scatter plot where each point is colored according to its cluster.
    Recently viewed products are marked as black crosses if provided.

    Returns:
    - fig (Figure): The Plotly figure object for the plot.
    """

    # Create the base cluster plot
    fig = px.scatter(
        pca_df,
        x='x',
        y='y',
        opacity=0.6,
        size_max=4,
        color= pca_df.cluster.astype(str),
        title='2D Visualization of Clusters with Recently Viewed Products',
        labels={'x': 'PCA Component 1', 'y': 'PCA Component 2'},
        category_orders={'cluster': list(range(num_clusters))},
        # show the product id in the tooltip
        hover_data={'product_id': True}

    )

    # Update layout to add legend title and adjust plot settings
    fig.update_layout(legend_title_text='Clusters', legend=dict(x=1, y=1), width=600, height=500)

    return fig

fig = plot_clusters(pca_df)
fig.show()

### Task 5 - Highlight recently viewed products

In [ ]:
print("The user has recently viewed the following products: ", recently_viewed_products)

Let's have a look at the records in our `clustered_data` dataframe related to the recently viewed products.

### Task 6 - Recommend products based on recently viewed products

Let's have a look at the recently viewed products titles

Let's see the distinct clusters of the recenetly viewed products.

Let's find the possible products for the recommendation.

Let's perform a groupby and generate a list of product IDs that can be recommended for each of the clusters.

In [ ]:
# write a python function to display the recommendations
def display_recommendations(row):
  # find the title of the product in df
  product_ids = row['random_recommendations']
  cluster = row.cluster

  titles = data. \
          filter(data["product_id"]. \
          isin(product_ids)).select("title").collect()

  print("\n")
  print("Recommendations for Cluster:", cluster)
  for title in titles:
    print(title[0])

recommendations_df.apply(display_recommendations, axis=1)